In [1]:
! pip install semantic-kernel -U
! pip install qdrant_client -U

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore

In [3]:
kernel = sk.Kernel()

In [4]:
chat_service_id = "default"

embed_service_id="embedding"

oai_chat_service = AzureChatCompletion(
        service_id=chat_service_id,
        env_file_path=".env"
)
embedding_gen = AzureTextEmbedding(
        service_id=embed_service_id,
        env_file_path=".env"
)



kernel.add_service(oai_chat_service)
kernel.add_service(embedding_gen)



# kernel.add_chat_service("azure_chat_competion_service", AzureChatCompletion("Your Azure OpenAI Service gpt-35-turbo-16k Model Delployment Name",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))


# kernel.add_text_embedding_generation_service(
#         "embeddings_services", AzureTextEmbedding("Your Azure OpenAI Service text-embedding-ada-002 Model Delployment Name", endpoint,api_key=api_key,api_version = "2023-12-01-preview")
# )

In [5]:
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory

In [6]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection('aboutMe')

In [7]:
# kernel.register_memory_store(memory_store=qdrant_store)

# kernel.add_plugin(QdrantMemoryPlugin(qdrant_store), "TextMemoryPlugin")


memory = SemanticTextMemory(storage=qdrant_store, embeddings_generator=AzureTextEmbedding( env_file_path=".env"))
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

KernelPlugin(name='TextMemoryPlugin', description=None, functions={'recall': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='recall', plugin_name='TextMemoryPlugin', description='Recall a fact from the long term memory', parameters=[KernelParameterMetadata(name='ask', description='The information to retrieve', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The information to retrieve'}, function_schema_include=True), KernelParameterMetadata(name='collection', description='The collection to search for information.', default_value='generic', type_='str', is_required=False, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The collection to search for information.'}, function_schema_include=True), KernelParameterMetadata(name='relevance', description='The relevance score, from 0.0 to 1.0; 1.0 means perfect match', default_value=0.75, type_='float', is_required=False, type_o

In [8]:
base_vectordb = 'aboutMe'

In [9]:

async def populate_memory(memory: SemanticTextMemory) -> None:
    # Add some documents to the semantic memory
    # await memory.save_information(collection=collection_id, id="info1", text="Your budget for 2024 is $100,000")
    # await memory.save_information(collection=collection_id, id="info2", text="Your savings from 2023 are $50,000")
    # await memory.save_information(collection=collection_id, id="info3", text="Your investments are $80,000")
    await memory.save_information(base_vectordb, id="info1", text="Kinfey is Microsoft Cloud Advocate")
    await memory.save_information(base_vectordb, id="info2", text="Kinfey is ex-Microsoft MVP")
    await memory.save_information(base_vectordb, id="info3", text="Kinfei is AI Expert")
    await memory.save_information(base_vectordb, id="info4", text="OpenAI is a company that is developing artificial general intelligence (AGI) with widely distributed economic benefits.")


In [10]:
await populate_memory(memory)

In [11]:
ask = "who is kinfey？"

memories = await memory.search(base_vectordb, ask,limit=3, min_relevance_score=0.77)

In [12]:
i = 0
for memory in memories:
    i = i + 1
    print(f"Top {i} Result: {memory.text} with score {memory.relevance}")

Top 1 Result: Kinfey is ex-Microsoft MVP with score 0.86921096
Top 2 Result: Kinfey is Microsoft Cloud Advocate with score 0.8400178
Top 3 Result: Kinfei is AI Expert with score 0.8256961
